In [6]:
import numpy as np
from packages.Utility.CNN.Callbacks import *
from packages.Utility.DatasetManager import DatasetReader
from packages.Utility.Gabor import GaborFilterBanks, ComplexGaborParams, GaborParams
from packages.Model.AlexNet import AlexNet1

# Inisialisasi / Persiapan

In [7]:
fold_index = 0
dataset_dir = 'dataset/bin/GH.227.B.5K'
trained_dir = 'trained/GH.227.B.5K AN1.1'

# Memuat Dataset

In [8]:
reader = DatasetReader(dataset_dir)

X_train, Y_train_onehot, Y_train_single = reader.getTrainData(fold_index)
X_test, Y_test_onehot, Y_test_single = reader.getTestData(fold_index)

# Inisiasi Callbacks

In [9]:
val_model = ValidateModel(X_train, Y_train_onehot)
save_model = SaveModel(val_model, base_dir = trained_dir, fold = fold_index + 1, min_acc = 0.1)
stop_train = StopTraining(val_model)
history_save = HistorySaver(val_model, base_dir = trained_dir, fold = fold_index + 1)

# Parameter Gabor

In [10]:
class ChannelizeGaborParams(GaborParams):
    
    def getParams(self, shape, i, j):
        multiplier = 360 / shape[3]
        degree = multiplier * i
        
        i += 1
        j += 1
        
        lambd = (i * j) / shape[2]
        theta = degree * np.pi / 180
        psi = (i + j) / shape[2]
        sigma = (i + j)
        gamma = (i + j) / shape[2]
        
        return lambd, theta, psi, sigma, gamma

# Mendefinisikan Kernel Gabor

In [ ]:
kernel1 = GaborFilterBanks(ComplexGaborParams(
    psi = (1, 48), gamma = (0.75, 1), lambd = (3, 5), sigma = (2, 3)
))

kernel2 = GaborFilterBanks(ComplexGaborParams(
    psi = (1, 128), gamma = (0.05, 0.75), sigma = (1, 10), lambd = (5, 10)
))

kernel3 = GaborFilterBanks(ComplexGaborParams(
    psi = (1, 192), gamma = (0.1, 0.75), sigma = (1, 10), lambd = (5, 10)
))

kernel4 = GaborFilterBanks(ComplexGaborParams(
    psi = (1, 192), gamma = (0.05, 0.75), sigma = (1, 10), lambd = (5, 10)
))

kernel5 = GaborFilterBanks(ComplexGaborParams(
    psi = (1, 128), gamma = (0.05, 0.75), sigma = (1, 10), lambd = (5, 10)
))

# Mendefinisikan Model

In [ ]:
ars = AlexNet1(kernels = {
    1: kernel1,
    2: kernel2,
    3: kernel3,
    4: kernel4,
    5: kernel5
}, loss = 'categorical_crossentropy', optimizer = 'adadelta')

model = ars.model
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c1 (Conv2D)                  (None, 55, 55, 96)        34944     
_________________________________________________________________
mp1 (MaxPooling2D)           (None, 27, 27, 96)        0         
_________________________________________________________________
c2 (Conv2D)                  (None, 23, 23, 256)       614656    
_________________________________________________________________
mp2 (MaxPooling2D)           (None, 11, 11, 256)       0         
_________________________________________________________________
c3 (Conv2D)                  (None, 11, 11, 384)       885120    
_________________________________________________________________
c4 (Conv2D)                  (None, 11, 11, 384)       1327488   
_________________________________________________________________
c5 (

In [ ]:
ars.save(f'{trained_dir}/Fold {fold_index + 1}/init.h5')

# Pelatihan

In [ ]:
ars.compile()
history = model.fit(X_train, Y_train_onehot, epochs = 30, verbose = 1, batch_size = 128, callbacks=[val_model, save_model, history_save, stop_train], initial_epoch = 0)